In [167]:
import SimFunctions
import SimClasses
import SimRNG
import simpy
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.stats import probplot, kstest
import pickle
from scipy.stats.distributions import norm, expon, erlang, lognorm, weibull_min, gamma
import statsmodels.api as sm
warnings.filterwarnings("ignore")
# fix random number seed
np.random.seed(1)


# Classes

In [168]:
class Station:
    def __init__(self, station_id, level, capacity):
        self.id = station_id
        self.level = level
        self.capacity = capacity
        self.bikes = {}
        self.bike_list = []
        for i in range(level):
            bike_id = f"{station_id}-{i+1}"  # create unique bike ID
            self.bikes[bike_id] = True  # mark bike as available
            self.bike_list.append(bike_id)
    def rent_bike(self):
        # Request a bike from the station
        for bike_id, available in self.bikes.items():
            if available:
                self.bikes[bike_id] = False  # mark bike as rented
                self.level -= 1
                self.bike_list.remove(bike_id)
                return bike_id
        return None

    def return_bike(self, bike_id):
        # # Return a bike to the station
        # self.bikes[bike_id] = True  # mark bike as available
        # self.level += 1
        self.bike_list.append(bike_id)
        self.level += 1
        
        if bike_id in self.bikes.keys() and not self.bikes[bike_id]:
            self.bikes[bike_id] = True  # mark bike as available
            # self.level += 1
            # self.bike_list.append(bike_id)
            return True
        return False
    
    def Get_Bike_List(self):
        return self.bike_list
    
class Customer:
    def __init__(self, customer_id, start_station, bike=None):
        self.customer_id = customer_id
        self.start_station = start_station
        self.end_id = 0
        self.station_level = 0
        self.bike = bike

    def rent_bike(self):
        station = StationDict[self.start_station]
        self.station_level = station.level -1
        self.bike = station.rent_bike()
        print("(CUSTOMER ARRIVAL) Customer ID: {} | Bike ID {} | Station ID: {}".format(self.customer_id, self.bike, self.start_station))
        SimFunctions.Schedule(Calendar, "Departure", 0.5)#2*SimRNG.Lognormal(0.1, np.sqrt(0.066), 6))
        
    def return_bike(self, end_station, bike):
        destination_station = StationDict[end_station]
        self.station_level = destination_station.level
        destination_station.return_bike(bike)
        
# Initialize the stations and create a dictionary mapping station IDs to Station instances
Stations = [Station(station_id=1, level=7, capacity=10), Station(station_id=2, level=15, capacity=20)]
StationDict = {station.id: station for station in Stations}

# Functions

In [169]:
ZSimRNG = SimRNG.InitializeRNSeed()
Calendar = SimClasses.EventCalendar()
TheQueues = []
TheCTStats = []
TheDTStats = []
TheResources = []

In [170]:
def NextCustomerID():
    if not hasattr(NextCustomerID, "counter"):
        NextCustomerID.counter = 0
    NextCustomerID.counter += 1
    return NextCustomerID.counter

def Customer_Arrival_Rate(station_id):
    U = SimRNG.Uniform(0, 1, 5)
    if U < (1/3):
        station_id = 1
    elif U < (2/3) and U > (1/3):
        station_id = 2
    else:
        station_id = 3
    return station_id

def Customer_Arrival(station_id, empty_error, CustomerList):
    SimFunctions.Schedule(Calendar, "Customer_Arrival", SimRNG.Expon(0.5, 1))
    station = StationDict[station_id]
        
    # Station is empty
    if station.level <= 0:
        print(f"    EMPTY -- Customer Arrives at STATION {station.id} -- EMPTY")
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
        empty_error += 1
    else:
        customer_id = NextCustomerID()
        customer = Customer(customer_id, station_id)
        CustomerList.append(customer)
        customer.start_station = station_id
        customer.rent_bike()

    return Customer_Arrival_Rate(station_id), empty_error, customer

def Bike_Arrival(end_id, full_error, CustomerList):
    last_customer = CustomerList[-1]
    last_customer.end_id = end_id
    destination_station = StationDict[end_id]
    #bike = destination_station
    for customer in CustomerList:
        if customer.end_id == end_id and customer.bike is not None:
            print("(BIKE RETURNED) Customer ID: {} | Bike ID: {} | From: {} | To: {} w/ Level: {}".format(customer.customer_id, customer.bike, customer.start_station, customer.end_id, destination_station.level))
            print("-------------------------------------------------------------------------")
            destination_station.return_bike(customer.bike)
            #customer.return_bike(end_id, customer.bike)
            CustomerList.remove(customer)
            return full_error
    #print(f"    BIKE NOT FOUND -- Bike Arrives at STATION {destination_station.id}")
    if destination_station.level >= destination_station.capacity:
        print(f"    FULL -- STATION {destination_station.id} FULL")
        full_error += 1
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
    return full_error
    
    # Station full or bike not found, schedule a retrial event
    print(f"    FULL -- Bike Arrives at STATION {destination_station.id} -- FULL")
    SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
    return full_error + 1


def Retrial(end_id, Full_Error, bike):
    print("Retrial")
    destination_station = StationDict[end_id]
    # Destination Station is not full
    if destination_station.level < destination_station.capacity:
        destination_station.return_bike(bike)
        print("     Bike has succesfully retried and returned bike to Station",
              destination_station.id)
    else:
        # STATION FULL
        print(f"    STATION {destination_station.id} FULL")
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
        Full_Error += 1
    return Full_Error


def Departure(station_id, customer):
    station = StationDict[station_id]
    print("(DEPARTING) Station ID: {} | Customer ID: {} | Level {} | ".format(customer.start_station, customer.customer_id, customer.station_level))

    #print("(DEPARTING) Station ID: {} | Customer ID: {} | Level {} | ".format(station.id, customer.customer_id, station.level))
    print("-------------------------------------------------------------------------")
    end_id = Destination(station)
    customer.end_id = end_id
    SimFunctions.Schedule(Calendar, "Bike_Arrival", 2 *SimRNG.Lognormal(0.1, np.sqrt(0.066), 4))
    return end_id


def Destination(station):
    #station = StationDict[station_id]
    station_id = station.id
    U = SimRNG.Uniform(0, 1, 3)
    if U < (1/3):
        end_id = 1
    elif U < (2/3) and U > (1/3):
        end_id = 2
    else:
        end_id = 3
    #end_id = 2 if U > 1/2 else 1

    end_station = StationDict[end_id]
    print("(DESTINATION) Station ID: {} | Level: {}".format(end_station.id, end_station.level))
    #print("-------------------------------------------------------------------------")
    return end_id


# Simulation

In [171]:
ZSimRNG = SimRNG.InitializeRNSeed()

Calendar = SimClasses.EventCalendar()

Queue = SimClasses.FIFOQueue()
Wait = SimClasses.DTStat()
Bikes = SimClasses.Resource()

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

# Stations = [7, 15]
# Station_Capacity = [10, 20]

Station_1_Resource = []
Station_2_Resource = []
CustomerList = []

MeanTBA = 0.5
MeanTR = 0.5
MeanST = 0.5

Phases = 3
RunLength = 50
WarmUp = 5
end_id = 1

for reps in range(0, 1, 1):
    Full_Error = 0
    Empty_Error = 0 
    
    inital_station_id = np.random.random_integers(1, 3)
    station_id = inital_station_id
    
    # Initialize the stations and create a dictionary mapping station IDs to Station instances
    Stations = [Station(station_id=1, level=15, capacity=20), Station(station_id=2, level=35, capacity=40), Station(station_id=3, level=24, capacity=25)]
    StationDict = {station.id: station for station in Stations}

    SimFunctions.SimFunctionsInit(Calendar, TheQueues, TheCTStats, TheDTStats, TheResources)
    SimFunctions.Schedule(Calendar, "Customer_Arrival", SimRNG.Expon(MeanTBA, 1))

    NextEvent = Calendar.Remove()
    SimClasses.Clock = NextEvent.EventTime
    if NextEvent.EventType == "Customer_Arrival":
        Customer_Arrival(inital_station_id, Empty_Error, CustomerList)


    for hours in [8, 8.3]:#, 9, 9.3, 10, 10.3, 11, 11.3, 12]:  # 30 min intervals
        print(f"Clock Hour | {hours}")
        for minutes in [i for i in range(1, 10)]:  # 30 min intervals
            print("Minute | {}".format(minutes))
            NextEvent = Calendar.Remove()
            SimClasses.Clock = NextEvent.EventTime
            
            if NextEvent.EventType == "Customer_Arrival":
                station_id, Empty_Error, customer = Customer_Arrival(station_id, Empty_Error, CustomerList)

            elif NextEvent.EventType == "Departure":
                end_id = Departure(station_id, customer)
            
            elif NextEvent.EventType == "Bike_Arrival":
                Full_Error = Bike_Arrival(end_id, Full_Error, CustomerList)
            
            elif NextEvent.EventType == "Retrial":
                Full_Error = Retrial(end_id, Full_Error, bike)
               
    # print(StationDict[1].bikes)
    # print(StationDict[2].bikes)
    # print(StationDict[3].bikes)
    # print()
    print(StationDict[1].Get_Bike_List())
    print(StationDict[2].Get_Bike_List())
    print(StationDict[3].Get_Bike_List())
    print("Num of Full Errors:", Full_Error)
    print("Num of Empty Errors:", Empty_Error)

(CUSTOMER ARRIVAL) Customer ID: 1 | Bike ID 2-1 | Station ID: 2
Clock Hour | 8
Minute | 1
(CUSTOMER ARRIVAL) Customer ID: 2 | Bike ID 2-2 | Station ID: 2
Minute | 2
(CUSTOMER ARRIVAL) Customer ID: 3 | Bike ID 1-1 | Station ID: 1
Minute | 3
(DEPARTING) Station ID: 1 | Customer ID: 3 | Level 14 | 
-------------------------------------------------------------------------
(DESTINATION) Station ID: 2 | Level: 33
Minute | 4
(BIKE RETURNED) Customer ID: 3 | Bike ID: 1-1 | From: 1 | To: 2 w/ Level: 33
-------------------------------------------------------------------------
Minute | 5
(CUSTOMER ARRIVAL) Customer ID: 4 | Bike ID 1-2 | Station ID: 1
Minute | 6
(DEPARTING) Station ID: 1 | Customer ID: 4 | Level 13 | 
-------------------------------------------------------------------------
(DESTINATION) Station ID: 1 | Level: 13
Minute | 7
(DEPARTING) Station ID: 1 | Customer ID: 4 | Level 13 | 
-------------------------------------------------------------------------
(DESTINATION) Station ID: 3 

In [172]:
# def Departure(station_id):
#     station = StationDict[station_id]
#     print("(DEPARTING) Station ID: {} | Level {}".format(
#         station.id, station.level, station.capacity))
#     print("-------------------------------------------------------------------------")

#     end_id = Destination(station)
#     SimFunctions.Schedule(Calendar, "Bike_Arrival", 2 *
#                           SimRNG.Lognormal(0.1, np.sqrt(0.066), 4))

#     # Rent a bike from the station
#     station.rent_bike()
#     return end_id


# def Destination(station_id):
#     # print("Destination")
#     #station = StationDict[station_id]

#     U = SimRNG.Uniform(0, 1, 3)
#     end_id = 2 if U > 1/2 else 1

#     end_station = StationDict[end_id]
#     print("(DESTINATION) Station ID: {} | Level: {}".format(
#         end_station.id, end_station.level))
#     print("-------------------------------------------------------------------------")
#     return end_id


# def Customer_Arrival_Rate(station_id):
#     U = SimRNG.Uniform(0, 1, 3)
#     station_id = 2 if U > 1/2 else 1
#     return station_id


# def Customer_Arrival(station_id, Empty_Error, bike):
#     SimFunctions.Schedule(Calendar, "Customer_Arrival", SimRNG.Expon(0.5, 1))
#     station = StationDict[station_id]

#     new_station_id = Customer_Arrival_Rate(station_id)

#     # Station is empty
#     if station.level <= 0:
#         print(f"    Customer Arrives at STATION {station.id} -- EMPTY")
#         SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
#         Empty_Error += 1
#     else:
#         bike = station.rent_bike()
#         if bike is None:
#             print(f"    Customer Arrives at STATION {station.id} -- EMPTY")
#             SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(1, 5))
#             Empty_Error += 1
#         else:
#             print("(CUSTOMER ARRIVAL) Station ID: {} | Level: {} | Capacity: {} | Bike ID {}".format(
#                 station.id, station.level, station.capacity, bike))
#             print(
#                 "-------------------------------------------------------------------------")
#             SimFunctions.Schedule(Calendar, "Departure",
#                                   2*SimRNG.Lognormal(0.1, np.sqrt(0.066), 6))

#     return new_station_id, Empty_Error, bike


# def Bike_Arrival(end_id, Full_Error, bike):
#     destination_station = StationDict[end_id]
#     # Destination Station is not full
#     if destination_station.level < destination_station.capacity:
#         destination_station.return_bike(bike)
#         print("(BIKE ARRIVAL) Station ID: {} | Level: {} | Bike ID: {}".format(
#             destination_station.id, destination_station.level, bike))
#         print("-------------------------------------------------------------------------")
#     else:
#         # STATION FULL
#         print(f"    Bike Arrives at STATION {destination_station.id} -- FULL")
#         SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
#         Full_Error += 1
#     return Full_Error


# def Retrial(end_id, Full_Error, bike):
#     print("Retrial")
#     destination_station = StationDict[end_id]
#     # Destination Station is not full
#     if destination_station.level < destination_station.capacity:
#         destination_station.return_bike(bike)
#         print("     Bike has succesfully retried and returned bike to Station",
#               destination_station.id)
#     else:
#         # STATION FULL
#         print(f"    STATION {destination_station.id} FULL")
#         SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
#         Full_Error += 1
#     return Full_Error


# def NextCustomerID():
#     if not hasattr(NextCustomerID, "counter"):
#         NextCustomerID.counter = 0
#     NextCustomerID.counter += 1
#     return NextCustomerID.counter


In [166]:
# ZSimRNG = SimRNG.InitializeRNSeed()

# Calendar = SimClasses.EventCalendar()

# Queue = SimClasses.FIFOQueue()
# Wait = SimClasses.DTStat()
# Bikes = SimClasses.Resource()

# TheCTStats = []
# TheDTStats = []
# TheQueues = []
# TheResources = []

# # Stations = [7, 15]
# # Station_Capacity = [10, 20]

# Station_1_Resource = []
# Station_2_Resource = []

# MeanTBA = 0.5
# MeanTR = 0.5
# MeanST = 0.5

# Phases = 3
# RunLength = 50
# WarmUp = 5
# end_id = 1

# for reps in range(0, 1, 1):
#     Full_Error = 0
#     Empty_Error = 0

#     inital_station_id = np.random.random_integers(1, 2)
#     station_id = inital_station_id

#     # Initialize the stations and create a dictionary mapping station IDs to Station instances
#     Stations = [Station(station_id=1, level=15, capacity=20),
#                 Station(station_id=2, level=35, capacity=40)]
#     StationDict = {station.id: station for station in Stations}

#     SimFunctions.SimFunctionsInit(
#         Calendar, TheQueues, TheCTStats, TheDTStats, TheResources)
#     SimFunctions.Schedule(Calendar, "Customer_Arrival",
#                           SimRNG.Expon(MeanTBA, 1))

#     NextEvent = Calendar.Remove()
#     SimClasses.Clock = NextEvent.EventTime
#     if NextEvent.EventType == "Customer_Arrival":
#         Customer_Arrival(inital_station_id, Empty_Error, 0)

#     for hours in [8, 8.3]:  # , 9, 9.3, 10, 10.3, 11, 11.3, 12]:  # 30 min intervals
#         print(f"Clock Hour | {hours}")
#         for minutes in [i for i in range(1, 31)]:  # 30 min intervals
#             print("Minute | {}".format(minutes))
#             NextEvent = Calendar.Remove()
#             SimClasses.Clock = NextEvent.EventTime

#             if NextEvent.EventType == "Customer_Arrival":
#                 station_id, Empty_Error, bike = Customer_Arrival(
#                     station_id, Empty_Error, bike)

#             elif NextEvent.EventType == "Departure":
#                 end_id = Departure(station_id)

#             elif NextEvent.EventType == "Bike_Arrival":
#                 Full_Error = Bike_Arrival(end_id, Full_Error, bike)

#             elif NextEvent.EventType == "Retrial":
#                 Full_Error = Retrial(end_id, Full_Error, bike)

#     print(StationDict[1].bikes)
#     print(StationDict[2].bikes)
#     print("Num of Full Errors:", Full_Error)
#     print("Num of Empty Errors:", Empty_Error)
